In [5]:
import pandas as pd
import numpy as np
import cv2
import os
import re
from scipy.optimize import linear_sum_assignment

In [6]:
bodiesDF = pd.read_pickle('synchronization/synchronize')
facesDF = pd.read_pickle('faces')
uniqueNames = np.load('uniqueName.npy')

In [7]:
noneList = [None] * facesDF.embs.iloc[0].shape[0]
noneArray = [noneList] * bodiesDF.shape[0]
len(bodiesDF['trackX'])
len(noneArray)

1728

In [8]:
bodiesDF['blur'] = np.nan
bodiesDF['embedding'] =  noneArray

In [9]:
bodiesDF

traSeq  trackId  trackX  trackY  trackW  trackH    height  distance  \
seq  id                                                                        
4793 4     4793        4   326.4   124.0   387.6   458.0  1.571027  2.248792   
     5     4793        5   802.4   150.0   442.0   520.0  1.519628  1.882822   
     6     4793        6  1135.6   108.0   309.4   364.0  1.475796  2.760290   
     7     4793        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4794 4     4794        4   323.0   124.0   387.6   456.0  1.569231  2.253340   
     5     4794        5   805.8   152.0   438.6   518.0  1.519507  1.894786   
     6     4794        6  1128.8   104.0   312.8   370.0  1.486020  2.723061   
     7     4794        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4795 4     4795        4   316.2   124.0   387.6   458.0  1.569074  2.248418   
     5     4795        5   805.8   154.0   438.6   516.0  1.520249  1.892176   
     6     4795        6  1125.4   104.0   316.2   374.0  1.490240  2.710115   
     7     4795        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4796 4     4796        4   306.0   124.0   387.6   458.0  1.567878  2.250316   
     5     4796        5   809.2   152.0   438.6   516.0  1.524655  1.898137   
     6     4796        6  1125.4   104.0   316.2   374.0  1.490240  2.710115   
     7     4796        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4797 4     4797        4   306.0   124.0   387.6   458.0  1.565820  2.250340   
     5     4797        5   816.0   156.0   435.2   514.0  1.521350  1.894691   
     6     4797        6  1125.4   104.0   316.2   374.0  1.490240  2.710115   
     7     4797        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4798 4     4798        4   299.2   124.0   387.6   458.0  1.566764  2.238508   
     5     4798        5   822.8   158.0   435.2   512.0  1.518310  1.894372   
     6     4798        6  1115.2   100.0   323.0   382.0  1.492517  2.683039   
     7     4798        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4799 4     4799        4   295.8   124.0   391.0   460.0  1.566956  2.234768   
     5     4799        5   833.0   160.0   431.8   508.0  1.515446  1.899774   
     6     4799        6  1115.2   100.0   323.0   382.0  1.492517  2.683039   
     7     4799        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4800 4     4800        4   299.2   124.0   391.0   460.0  1.563260  2.241266   
     5     4800        5   836.4   160.0   431.8   508.0  1.512625  1.898493   
...         ...      ...     ...     ...     ...     ...       ...       ...   
5173 11    5173       11   656.2   114.0    95.2   112.0  1.076667  6.554167   
     12    5173       12   234.6   168.0   394.4   466.0  1.492208  2.148317   
5174 5     5174        5  1009.8   116.0   363.8   428.0  1.485402  2.264647   
     6     5174        6    -3.4    86.0   292.4   344.0  1.451042  3.188060   
     9     5174        9   642.6   128.0   268.6   318.0  1.516538  3.130712   
     10    5174       10    81.6   142.0   438.6   516.0  1.578404  2.191300   
     11    5174       11   656.2   114.0    95.2   112.0  1.076667  6.554167   
     12    5174       12   227.8   168.0   387.6   458.0  1.493150  2.222069   
5175 5     5175        5  1013.2   118.0   363.8   430.0  1.483078  2.226244   
     6     5175        6    -3.4    86.0   292.4   344.0  1.451042  3.188060   
     9     5175        9   649.4   124.0   268.6   318.0  1.514305  3.104354   
     10    5175       10    81.6   142.0   438.6   516.0  1.578404  2.191300   
     11    5175       11   656.2   114.0    95.2   112.0  1.076667  6.554167   
     12    5175       12   231.2   168.0   384.2   454.0  1.496295  2.241670   
5176 5     5176        5  1020.0   118.0   370.6   438.0  1.486080  2.173221   
     6     5176        6   -34.0    84.0   295.8   350.0  1.447993  3.162891   
     9     5176        9   659.6   122.0   272.0   320.0  1.512758  3.100690   
     10  

In [10]:
facesDF

,faceX,faceY,rgb,blurScore,embs
0,575.0,82.0,22.0,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,..."
1,1131.0,161.0,22.0,790.153174,"[0.0554316, 0.0860607, -0.00389893, 0.0320879,..."
2,1375.0,84.0,22.0,600.989129,"[-0.0656685, 0.047579, -0.100778, 0.00661445, ..."
3,575.0,82.0,23.0,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,..."
4,1131.0,161.0,23.0,790.153174,"[0.0554316, 0.0860607, -0.00389893, 0.0320879,..."
5,1375.0,84.0,23.0,600.989129,"[-0.0656685, 0.047579, -0.100778, 0.00661445, ..."
6,575.0,82.0,24.0,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,..."
7,1131.0,161.0,24.0,790.153174,"[0.0554316, 0.0860607, -0.00389893, 0.0320879,..."
8,1375.0,84.0,24.0,600.989129,"[-0.0656685, 0.047579, -0.100778, 0.00661445, ..."
9,575.0,82.0,25.0,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,..."


In [11]:
for seq in bodiesDF.rgbSeq.unique():
    bigDist = []
    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset='id', keep='first')
    faces = facesDF[facesDF.rgb == seq]
    
    for fi, f in faces.iterrows():
        faceCentroid = (int(f.faceX), int(f.faceY))
        distance = []
        if len(bodies) > 0:
            for bi, b in bodies.iterrows():
                upLeft = tuple((int(b.trackX), int(b.trackY)))
                downRight = tuple((int(b.trackX) + int(b.trackW), int(b.trackY) + int(b.trackH)))
                bodyCentroid = tuple((np.asarray(upLeft) + np.asarray(downRight)) / 2)
                
                inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackW)**2
                up = bodyCentroid[1] > faceCentroid[1]
                if inside and up:
                    dist = ((bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2)**(0.5)
                    distance.append(dist)
                else:
                    distance.append(2237)
        bigDist.append(distance)

    if len(bigDist) > 0:
        row_ind, col_ind = linear_sum_assignment(np.asarray(bigDist).T)
        for i in range(len(row_ind)):
            b = bodies.iloc[row_ind[i]]
            bodiesDF.loc.__setitem__( ((int(b.seq), int(b.id)), 'embedding'), 
                                     np.asarray([faces.iloc[col_ind[i]]['embs']]).T )
            bodiesDF.loc.__setitem__(((int(b.seq), int(b.id)), 'blur'), faces.iloc[col_ind[i]]['blurScore'])

In [12]:
bodiesDF

traSeq  trackId  trackX  trackY  trackW  trackH    height  distance  \
seq  id                                                                        
4793 4     4793        4   326.4   124.0   387.6   458.0  1.571027  2.248792   
     5     4793        5   802.4   150.0   442.0   520.0  1.519628  1.882822   
     6     4793        6  1135.6   108.0   309.4   364.0  1.475796  2.760290   
     7     4793        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4794 4     4794        4   323.0   124.0   387.6   456.0  1.569231  2.253340   
     5     4794        5   805.8   152.0   438.6   518.0  1.519507  1.894786   
     6     4794        6  1128.8   104.0   312.8   370.0  1.486020  2.723061   
     7     4794        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4795 4     4795        4   316.2   124.0   387.6   458.0  1.569074  2.248418   
     5     4795        5   805.8   154.0   438.6   516.0  1.520249  1.892176   
     6     4795        6  1125.4   104.0   316.2   374.0  1.490240  2.710115   
     7     4795        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4796 4     4796        4   306.0   124.0   387.6   458.0  1.567878  2.250316   
     5     4796        5   809.2   152.0   438.6   516.0  1.524655  1.898137   
     6     4796        6  1125.4   104.0   316.2   374.0  1.490240  2.710115   
     7     4796        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4797 4     4797        4   306.0   124.0   387.6   458.0  1.565820  2.250340   
     5     4797        5   816.0   156.0   435.2   514.0  1.521350  1.894691   
     6     4797        6  1125.4   104.0   316.2   374.0  1.490240  2.710115   
     7     4797        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4798 4     4798        4   299.2   124.0   387.6   458.0  1.566764  2.238508   
     5     4798        5   822.8   158.0   435.2   512.0  1.518310  1.894372   
     6     4798        6  1115.2   100.0   323.0   382.0  1.492517  2.683039   
     7     4798        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4799 4     4799        4   295.8   124.0   391.0   460.0  1.566956  2.234768   
     5     4799        5   833.0   160.0   431.8   508.0  1.515446  1.899774   
     6     4799        6  1115.2   100.0   323.0   382.0  1.492517  2.683039   
     7     4799        7   822.8   108.0    88.4   104.0  1.050643  6.857205   
4800 4     4800        4   299.2   124.0   391.0   460.0  1.563260  2.241266   
     5     4800        5   836.4   160.0   431.8   508.0  1.512625  1.898493   
...         ...      ...     ...     ...     ...     ...       ...       ...   
5173 11    5173       11   656.2   114.0    95.2   112.0  1.076667  6.554167   
     12    5173       12   234.6   168.0   394.4   466.0  1.492208  2.148317   
5174 5     5174        5  1009.8   116.0   363.8   428.0  1.485402  2.264647   
     6     5174        6    -3.4    86.0   292.4   344.0  1.451042  3.188060   
     9     5174        9   642.6   128.0   268.6   318.0  1.516538  3.130712   
     10    5174       10    81.6   142.0   438.6   516.0  1.578404  2.191300   
     11    5174       11   656.2   114.0    95.2   112.0  1.076667  6.554167   
     12    5174       12   227.8   168.0   387.6   458.0  1.493150  2.222069   
5175 5     5175        5  1013.2   118.0   363.8   430.0  1.483078  2.226244   
     6     5175        6    -3.4    86.0   292.4   344.0  1.451042  3.188060   
     9     5175        9   649.4   124.0   268.6   318.0  1.514305  3.104354   
     10    5175       10    81.6   142.0   438.6   516.0  1.578404  2.191300   
     11    5175       11   656.2   114.0    95.2   112.0  1.076667  6.554167   
     12    5175       12   231.2   168.0   384.2   454.0  1.496295  2.241670   
5176 5     5176        5  1020.0   118.0   370.6   438.0  1.486080  2.173221   
     6     5176        6   -34.0    84.0   295.8   350.0  1.447993  3.162891   
     9     5176        9   659.6   122.0   272.0   320.0  1.512758  3.100690   
     10  

In [13]:
bodiesDF.to_pickle('bodyFace')

In [17]:
# For checking
for seq in bodiesDF.rgbSeq.unique():
    #print seq
    bigDist = []
    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset='id', keep='first')
    faces = facesDF[facesDF.rgb == seq]
    
    for fi, f in faces.iterrows():
        faceCentroid = (int(f.faceX), int(f.faceY))
        distance = []
        if len(bodies) > 0:
            for bi, b in bodies.iterrows():
                upLeft = tuple((int(b.trackX), int(b.trackY)))
                downRight = tuple((int(b.trackX) + int(b.trackW), int(b.trackY) + int(b.trackH)))
                bodyCentroid = tuple((np.asarray(upLeft) + np.asarray(downRight)) / 2)
                
                inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackW)**2                
                up = bodyCentroid[1] > faceCentroid[1]
                if inside and up:
                    dist = ((bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2)**(0.5)
                    distance.append(dist)
                else:
                    distance.append(2237)
        bigDist.append(distance)

    if len(bigDist) > 0:
        row_ind, col_ind = linear_sum_assignment(np.asarray(bigDist).T)
        for i in range(len(row_ind)):
            line = cv2.imread("/home/sabrine/notebook/modified2/" + str(seq) + '.jpg')
                            
            for i in range(len(col_ind)):
                b = bodies.iloc[row_ind[i]]
                upLeft = tuple((int(b.trackX), int(b.trackY)))
                downRight = tuple((int(b.trackX) + int(b.trackW), int(b.trackY) + int(b.trackH)))
                bodyCentroid = tuple((np.asarray(upLeft) + np.asarray(downRight)) / 2)

                f = faces.iloc[col_ind[i]]
                faceCentroid = (int(f.faceX), int(f.faceY))
                
                inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackW)**2
                line = cv2.circle(line, faceCentroid, (int(b.trackW)), (65,65,65), thickness=3)
                up = bodyCentroid[1] > faceCentroid[1]
                if inside and up:
                    line = cv2.line(line, bodyCentroid, faceCentroid, (255,65,255), thickness=3)

            cv2.imwrite("/home/sabrine/notebook/modified3/" + str(seq) + ".jpg", line)

In [ ]:
'''seqM = []
idM = []
indexM = []

for seq in bodiesDF.rgbSeq.unique():
    #seq = 14
    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset='id', keep='first')
    faces = facesDF[facesDF.rgbSeq == seq]
    
    img = cv2.imread("/home/sabrine/notebook/modified2/" + str(seq) + ".jpg")
    
    for index, face in faces.iterrows():
        faceCentroid = (int(face.faceX), int(face.faceY))
        distance = []
        for i, b in bodies.iterrows():
            upLeft = tuple((b.trackX, b.trackY))
            downRight = tuple((b.trackX + b.trackW, b.trackY + b.trackH))
            bodyCentroid = tuple((np.asarray(upLeft) + np.asarray(downRight)) / 2)
            
            
            inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackH.astype(int)/2)**2
            up = bodyCentroid[1] > faceCentroid[1]
            if inside and up:
                dist = ((bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2)**(0.5)
                distance.append(dist)
                
                color = (255, 255, 64)
                thickness = 1
                lineType = 8 # will discorver what it means
                radius = b.trackH.astype(int)/2
                img = cv2.circle(img, faceCentroid, int(radius), color, thickness, lineType)
            else:
                distance.append(np.inf)
                
        argMin = np.argmin(np.asarray(distance))
        seqMin = int(bodies.iloc[argMin].seq)
        idMin = int(bodies.iloc[argMin].id)
        seqM.append(seqMin)
        idM.append(idMin)
        indexM.append(index)
        
                
    cv2.imwrite("/home/sabrine/notebook/modified/" + str(seq) + ".jpg", img)'''

In [ ]:
'''for seq in bodiesDF.rgbSeq.unique():
    #seq = 14
    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset='id', keep='first')
    faces = facesDF[facesDF.rgbSeq == seq]
    
    print bodies
    print '#####'
    
    img = cv2.imread("/home/sabrine/notebook/modified2/" + str(seq) + ".jpg")
    
    for index, face in faces.iterrows():
        faceCentroid = (int(face.faceX), int(face.faceY))
        
        upLeft = np.asarray([bodies.trackX, bodies.trackY])
        downRight = np.asarray([bodies.trackX + bodies.trackW, bodies.trackY + bodies.trackH])
        print upLeft
        print downRight
        print type(downRight)
        
        bodyCentroid = (upLeft[O].add(downRight)).divide(2)
            
        inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackH.astype(int)/2)**2
        up = bodyCentroid[1] > faceCentroid[1]
        if inside and up:
            color = (255, 255, 64)
            thickness = 1
            radius = b.trackH.astype(int)/2
            img = cv2.circle(img, faceCentroid, int(radius), color, thickness, lineType)'''

In [ ]:
'''for seq in bodiesDF.rgbSeq.unique():
    bodies = bodiesDF[bodiesDF.rgbSeq == seq]
    faces = facesDF[facesDF.rgbSeq == seq]
    
    #print 'bodies'
    #print bodies
    #print '####'
    #print 'faces'
    #print faces
    for i, f in faces.iterrows():
        if len(bodies) > 0 :        
            
            candidates = bodies[
                [((((bodies.trackX.astype(int) + bodies.trackW.astype(int)) / 2 ) - f.faceX)**2 + 
                (((bodies.trackY.astype(int) + bodies.trackH.astype(int)) / 2 ) - f.faceY)**2) < 
                 (bodies.trackH.astype(int)/2)**2] # 250 arbitrary picked 
                and 
                ((bodies.trackY.astype(int) + bodies.trackH.astype(int)) / 2 ) < f.faceY
            ]
            if not candidates.empty:
                print '******'
                print f
                print candidates
                print '******''''